# ElasticSearch Amazon SageMaker Blog
# Loading an embedding from Hugging Face into Elasticsearch

This code will show you how to load a supported embedding model from Hugging Face into an elasticsearch cluster in [Elastic Cloud](https://cloud.elastic.co/)

# Setup


## Install and import required python libraries

Elastic uses the [eland python library](https://github.com/elastic/eland) to download modesl from Hugging Face hub and load them into elasticsearch

In [25]:
pip -q install eland elasticsearch sentence_transformers transformers torch==1.11

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
triton 2.0.0 requires cmake, which is not installed.
triton 2.0.0 requires lit, which is not installed.
torchdata 0.6.0 requires torch>1.13, but you have torch 1.11.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [26]:
from pathlib import Path
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient

import getpass

## Configure elasticsearch authentication.
The recommended authentication approach is using the [Elastic Cloud ID](https://www.elastic.co/guide/en/cloud/current/ec-cloud-id.html) and a [cluster level API key](https://www.elastic.co/guide/en/kibana/current/api-keys.html)

You can use any method you wish to set the required credentials. We are using getpass in this example to prompt for credentials to avoide storing them in github.

In [27]:
es_cloud_id = getpass.getpass('Enter Elastic Cloud ID:  ')
es_user = getpass.getpass('Enter cluster username:  ')
es_pass = getpass.getpass('Enter cluster password:  ')

## Connect to Elastic Cloud

In [28]:
#es = Elasticsearch(cloud_id=es_cloud_id,
#                   api_key=(es_api_id, es_api_key)
#                   )
es = Elasticsearch(cloud_id=es_cloud_id,
                   basic_auth=(es_user, es_pass)
                   )
es.info() # should return cluster info

ObjectApiResponse({'name': 'instance-0000000000', 'cluster_name': 'e676bcbabc614a61b038a91a02b2c6dc', 'cluster_uuid': 'AlptJ-AASPGPzHy33ahRMQ', 'version': {'number': '8.9.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '8aa461beb06aa0417a231c345a1b8c38fb498a0d', 'build_date': '2023-07-19T14:43:58.555259655Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

# Load the model From Hugging Face into Elasticsearch
Here we specify the model id from Hugging Face. The easiest way to get this id is clicking the copy the model name icon next to the name on the model page.

When calling `TransformerModel` you specify the HF model id and the task type. You can try specifying `auto` and eland will attempt to determine the correct type from info in the model config. This is not always possible so a list of specific `task_type` values can be viewed in the following code:
[Supported values](https://github.com/elastic/eland/blob/15a300728876022b206161d71055c67b500a0192/eland/ml/pytorch/transformers.py#*L41*)

In [29]:
# Set the model name from Hugging Face and task type
# hf_model_id='sentence-transformers/all-distilroberta-v1'
hf_model_id = 'BAAI/bge-base-en'
tm = TransformerModel(hf_model_id, "text_embedding")

#set the modelID as it is named in Elasticsearch
es_model_id = tm.elasticsearch_model_id()

# Download the model from Hugging Face
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

# Load the model into Elasticsearch
ptm = PyTorchModel(es, es_model_id)
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config) # ETA 7:30, also too big for 


  0%|          | 0/105 [00:00<?, ? parts/s]

# Starting the Model

## View information about the model
This is not required but can be handy to get a model overivew

In [30]:
# List the in elasticsearch
m = MlClient.get_trained_models(es, model_id=es_model_id)
m.body

{'count': 1,
 'trained_model_configs': [{'model_id': 'baai__bge-base-en',
   'model_type': 'pytorch',
   'created_by': 'api_user',
   'version': '8.9.0',
   'create_time': 1691986436892,
   'model_size_bytes': 0,
   'estimated_operations': 0,
   'license_level': 'platinum',
   'description': "Model BAAI/bge-base-en for task type 'text_embedding'",
   'tags': [],
   'input': {'field_names': ['text_field']},
   'inference_config': {'text_embedding': {'vocabulary': {'index': '.ml-inference-native-000001'},
     'tokenization': {'bert': {'do_lower_case': True,
       'with_special_tokens': True,
       'max_sequence_length': 512,
       'truncate': 'first',
       'span': -1}}}},
   'location': {'index': {'name': '.ml-inference-native-000001'}}}]}

## Deploy the model
This will load the model on the ML nodes and start the process(es) making it available for the NLP task

In [31]:
s = MlClient.start_trained_model_deployment(es, model_id=es_model_id)
s.body

ConnectionTimeout: Connection timed out

## Verify the model started without issue
Should output -> {'routing_state': 'started'}

In [32]:
stats = MlClient.get_trained_models_stats(es, model_id=es_model_id)
stats.body['trained_model_stats'][0]['deployment_stats']['nodes'][0]['routing_state']

{'routing_state': 'started'}